In [1]:
# Import the PyTorch library for tensor computations and GPU support
import torch

# Import T5 model and tokenizer from the Transformers library
from transformers import T5ForConditionalGeneration,T5Tokenizer

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Load the pre-trained T5 model for conditional generation from the specified repository
model = T5ForConditionalGeneration.from_pretrained("Michau/t5-base-en-generate-headline")

# Load the corresponding tokenizer for the T5 model from the specified repository
tokenizer = T5Tokenizer.from_pretrained("Michau/t5-base-en-generate-headline")

# Move the model to the specified device (GPU or CPU) for computation
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [22]:
def generate_headline(sentence):

    # Format the input sentence by prepending "headline: " to it
    formatted_input_sentence = "headline: " + sentence

    # Encode the formatted input using the tokenizer, returning PyTorch tensors
    encoding = tokenizer.encode_plus(formatted_input_sentence , return_tensors="pt")

    # Extract the input IDs from the encoding and move them to the specified device (CPU or GPU)
    input_ids = encoding["input_ids"].to(device)

    # Extract the attention masks from the encoding and move them to the specified device
    attention_masks = encoding["attention_mask"].to(device)

    # Generate headlines using the model with specified parameters for beam search
    generated_headlines = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=64,      # Set the maximum length of the generated headlines
        num_beams=3,        # Use beam search with 3 beams for better quality
        early_stopping=True  # Stop early if all beams reach the end of sequence token
    )

    # Decode the generated headline from token IDs to a string and clean up unwanted tokens
    decoded_headline = tokenizer.decode(generated_headlines[0]).replace("<pad>", "").replace("</s>", "").strip()

    # Return the cleaned-up generated headline
    return decoded_headline

In [24]:
def sentence_analyzer(sentence):

  # Apply the 'generate_headline' function on each sentence of 'sentence_list' by 'map' function and create a list of keywords for each sentence
  headline_lists = list(map(generate_headline , sentences_list))

  # Create a dictionary to hold extracted keywords for each sentence
  headlines_dict = {}

  # Putting the number of each sentence as a key and its related headline as a value in the headlines_dict dictionary
  for i in range(len(sentences_list)):
    headlines_dict[f'sentence {i+1}']= headline_lists[i]

  # Returning headline of each sentence in dictionary format
  return headlines_dict

In [25]:
sentences_list = ['Very early yesterday morning, the United States President Donald Trump reported he and his wife First Lady Melania Trump tested positive for COVID-19. Officials said the Trumps 14-year-old son Barron tested negative as did First Family and Senior Advisors Jared Kushner and Ivanka Trump. Trump took to social media, posting at 12:54 am local time (0454 UTC) on Twitter, "Tonight, [Melania] and I tested positive for COVID-19. We will begin our quarantine and recovery process immediately. We will get through this TOGETHER!" Yesterday afternoon Marine One landed on the White House s South Lawn flying Trump to Walter Reed National Military Medical Center (WRNMMC) in Bethesda, Maryland.Reports said both were showing "mild symptoms". Senior administration officials were tested as people were informed of the positive test. Senior advisor Hope Hicks had tested positive on Thursday.Presidential physician Sean Conley issued a statement saying Trump has been given zinc, vitamin D, Pepcid and a daily Aspirin. Conley also gave a single dose of the experimental polyclonal antibodies drug from Regeneron Pharmaceuticals.According to official statements, Trump, now operating from the WRNMMC, is to continue performing his duties as president during a 14-day quarantine. In the event of Trump becoming incapacitated, Vice President Mike Pence could take over the duties of president via the 25th Amendment of the US Constitution. The Pence family all tested negative as of yesterday and there were no changes regarding Pences campaign events',

                  'Artificial intelligence (AI) is a wide-ranging tool that enables people to rethink how we integrate information, analyze data, and use the resulting insights to improve decision making—and already it is transforming every walk of life. In this report, Darrell West and John Allen discuss AIs application across a variety of sectors, address issues in its development, and offer recommendations for getting the most out of AI while still protecting important human values.',

                  'The cerebrum is the largest part of the brain, accounting for 85 percent of the organs weight. The distinctive, deeply wrinkled outer surface is the cerebral cortex. Its the cerebrum that makes the human brain—and therefore humans—so formidable. Animals such as elephants, dolphins, and whales actually have larger brains, but humans have the most developed cerebrum. Its packed to capacity inside our skulls, with deep folds that cleverly maximize the total surface area of the cortex.',

                  'football, game in which two teams of 11 players, using any part of their bodies except their hands and arms, try to maneuver the ball into the opposing teams goal. Only the goalkeeper is permitted to handle the ball and may do so only within the penalty area surrounding the goal. The team that scores more goals wins.'
                  ]

In [26]:
sentence_analyzer(sentences_list)

{'sentence 1': 'Trump and First Lady Melania Test Positive for COVID-19',
 'sentence 2': 'Artificial Intelligence (AI) - A Wide-Range Tool',
 'sentence 3': 'The Cerebrum - The Largest Part of the Brain',
 'sentence 4': 'Football - A Football Game'}